# XAI Fidelity Benchmark Notebook

It evaluates fidelity, runtime, and scalability of mainstream post-hoc explainers (**SHAP**, **LIME**) and our proposed pre-hoc approach (**LinearGEX**) across multiple short-text datasets (*MeOffendEs*, *HOPE*, *CheckWorthy*, *DANA*).

The notebook contains the experiments reported in **Section 4** of the manuscript, computing fidelity as $R^2$ against each model’s native output (decision margins for LinearSVC, logits for BERT, and local surrogate fit for LIME). Auxiliary metrics (MAE or STD) and efficiency measures (per-instance time, batch size, batch runtime) are also recorded. Results are exported in structured tables consistent with the quantitative benchmarks presented in **Table 5** of the paper.

In [ ]:
# -*- coding: utf-8 -*-

In [ ]:
# https://github.com/INGEOTEC/microtc
try:
    import microtc
except ImportError:
    !pip install microtc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.8 MB/s eta 0:00:00


In [ ]:
try:
    import lime
except ImportError:
    !pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=9e67a15640943dc54022bc0fa1b3478bbbbf0c1cc61cc08fb398c51a192ea32b
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [ ]:
import json
import joblib
import torch
import pandas as pd
import numpy as np
from microtc import TextModel
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import time
import shap
import numpy as np
from lime.lime_text import LimeTextExplainer
import torch
import shap
from lime.lime_text import LimeTextExplainer
import time
from scipy.special import softmax
import re

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Configuración
base_ds_path = '/content/drive/MyDrive/Colab Notebooks/LinearGEX/ds/'


# =========================
# Experimentos
# =========================
datasets = {
    "meoffendes": {
        "train": "meoffendes_21_es_2_train.json",
        "test": "meoffendes_21_es_2_test.json",
        "bert_model": "dccuchile/bert-base-spanish-wwm-cased",
        "num_labels": 2,
    },
    "hope": {
        "train": "hope_24_en_4_train.json",
        "test": "hope_24_en_4_test.json",
        "bert_model": "bert-base-uncased",
        "num_labels": 4,
    },
    "checkworthy": {
        "train": "checkworthy_24_de_2_train.json",
        "test": "checkworthy_24_de_2_test.json",
        "bert_model": "bert-base-german-dbmdz-uncased",
        "num_labels": 2,
    },
    "dana": {
        "train": "dana_25_es_2_train.json",
        "test": "dana_25_es_2_test.json",
        "bert_model": "dccuchile/bert-base-spanish-wwm-cased",
        "num_labels": 2,
    },
}

In [ ]:
# Verificar Datasets y estructura de directorios

for name, info in datasets.items():
    print(f"\n=== Dataset: {name} ===")

    train_path = f"{base_ds_path}{info['train']}"
    test_path = f"{base_ds_path}{info['test']}"

    X_train = pd.read_json(train_path, lines=True)#.sample(n=200, random_state=42)
    X_test = pd.read_json(test_path, lines=True)#.sample(n=50, random_state=42)

    print(X_train.head(2))

    print(f"Train: {X_train.shape}")
    print(f"Test: {X_test.shape}")

#!ls -la '/content/drive/MyDrive/Colab Notebooks/LinearGEX/ds/'    ç
base_ds_path = '/content/drive/MyDrive/Colab Notebooks/LinearGEX/ds/'
print(base_ds_path)
!ls -la "{base_ds_path}/pretrained/linearsvc"



=== Dataset: meoffendes ===
                                                text  klass
0  @USUARIO @USUARIO Que rico mamas el camote tie...      0
1  Ampliación: La salida a final de mes del juego...      0
Train: (3795, 2)
Test: (1265, 2)

=== Dataset: hope ===
                                                text  klass
0  Glad I didn't put this bullshit in my body. \n...      0
1  #USER# I was at this game. We lost, and I got ...      0
Train: (4953, 2)
Test: (1239, 2)

=== Dataset: checkworthy ===
                                                text  klass
0  Die Ausbreitung des Virus sei beschränkter als...      0
1    Zwar sei ein Anstieg der Zahlen zu verzeichnen.      0
Train: (800, 2)
Test: (224, 2)

=== Dataset: dana ===
                                                text  klass
0   atentos a las palabras del coronel Baños. La ...      0
1  @DavidRaventos68 Pots comprovar personalment c...      1
Train: (520, 2)
Test: (131, 2)
/content/drive/MyDrive/Colab Notebooks/LinearGE

In [ ]:
# =================
# Función para cargar datasets

def load_json_dataset(ds_info):
    train_path = f"{base_ds_path}{ds_info['train']}"
    test_path = f"{base_ds_path}{ds_info['test']}"

    X_train = pd.read_json(train_path, lines=True)#.sample(n=200, random_state=42)
    X_test = pd.read_json(test_path, lines=True)#.sample(n=50, random_state=42)

    return X_train["text"].astype(str).tolist(), X_train["klass"].tolist(), X_test["text"].astype(str).tolist(), X_test["klass"].tolist()

In [ ]:
# =========================
# Función auxiliar: métricas

def compute_metrics(y_true, y_pred, average="macro"):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average=average, zero_division=0),
        "recall": recall_score(y_true, y_pred, average=average, zero_division=0),
        "f1": f1_score(y_true, y_pred, average=average, zero_division=0),
    }

In [ ]:
results = []

In [ ]:
# =========================
# Entrenamiento LinearSVC

def train_linearsvc(ds_name, ds_info):
    # Cargar datos
    X_train, y_train, X_test, y_test = load_json_dataset(ds_info)

    # Vectorización TF-IDF (µTC con q-grams)
    vectorizer = TextModel(token_list=[2, 3, 4], del_diac=True,  del_dup=False, lc=True, hashtag_option=None).fit(X_train)
    # ToDo entreno con X_train, y vectorizo ambos con X_train. X_test podria tener tokens no considerados
    X_train_vec, X_test_vec = vectorizer.transform(X_train), vectorizer.transform(X_test)

    # Entrenar modelo
    clf = LinearSVC() # ToDo parametros del LinearSVC
    clf.fit(X_train_vec, y_train)

    # Predicción
    y_pred = clf.predict(X_test_vec)
    metrics = compute_metrics(y_test, y_pred)

    # Guardar modelo y vectorizador
    joblib.dump(clf, f"{base_ds_path}pretrained/linearsvc/{ds_name}_linearsvc_model.pkl")
    joblib.dump(vectorizer, f"{base_ds_path}pretrained/linearsvc/{ds_name}_tfidf_mtc.pkl")

    return metrics

In [ ]:
recalculate = False

In [ ]:
# =========================
# Loop LinearSVC
# El modelo entrenado se guarda para reutilizarlo.

if recalculate:

    for ds_name, ds_info in datasets.items():
        print(f"\n=== Dataset: {ds_name} === {ds_info}")

        metrics_svc = train_linearsvc(ds_name, ds_info)
        print("LinearSVC:", metrics_svc)

        results.append((ds_name, "LinearSVC", metrics_svc))



=== Dataset: meoffendes === {'train': 'meoffendes_21_es_2_train.json', 'test': 'meoffendes_21_es_2_test.json', 'bert_model': 'dccuchile/bert-base-spanish-wwm-cased', 'num_labels': 2}
LinearSVC: {'accuracy': 0.783399209486166, 'precision': 0.7586934773909564, 'recall': 0.6919439792876569, 'f1': 0.7091963782318038}

=== Dataset: hope === {'train': 'hope_24_en_4_train.json', 'test': 'hope_24_en_4_test.json', 'bert_model': 'bert-base-uncased', 'num_labels': 4}
LinearSVC: {'accuracy': 0.6602098466505246, 'precision': 0.5765998537779892, 'recall': 0.5130118266612733, 'f1': 0.5323188848122438}

=== Dataset: checkworthy === {'train': 'checkworthy_24_de_2_train.json', 'test': 'checkworthy_24_de_2_test.json', 'bert_model': 'bert-base-german-dbmdz-uncased', 'num_labels': 2}
LinearSVC: {'accuracy': 0.7455357142857143, 'precision': 0.7049290372075182, 'recall': 0.6967228205836326, 'f1': 0.7003309315370714}

=== Dataset: dana === {'train': 'dana_25_es_2_train.json', 'test': 'dana_25_es_2_test.json'

In [ ]:
# =========================
# Entrenamiento BERT

def train_bert(ds_name, ds_info):
    # Cargar datos
    X_train, y_train, X_test, y_test = load_json_dataset(ds_info)

    # Crear datasets
    train_ds = Dataset.from_dict({"text": X_train, "label": y_train})
    test_ds = Dataset.from_dict({"text": X_test, "label": y_test})

    # Tokenizador y modelo
    tokenizer = AutoTokenizer.from_pretrained(ds_info['bert_model'])

    def tokenize(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True)

    train_ds = train_ds.map(tokenize, batched=True)
    test_ds = test_ds.map(tokenize, batched=True)

    model = AutoModelForSequenceClassification.from_pretrained(ds_info['bert_model'], num_labels=ds_info['num_labels'])

    # Entrenamiento
    training_args = TrainingArguments(
        output_dir=f"{base_ds_path}pretrained/bert/{ds_name}_bert",
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    def compute_metrics_trainer(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        return compute_metrics(labels, preds)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_trainer,
    )

    trainer.train()
    results = trainer.evaluate()

    # Guardar modelo y tokenizador
    model.save_pretrained(f"{base_ds_path}pretrained/bert/{ds_name}_bert")
    tokenizer.save_pretrained(f"{base_ds_path}pretrained/bert/{ds_name}_bert")

    return results

In [ ]:
# =========================
# Loop LinearSVC

if recalculate:
    for ds_name, ds_info in datasets.items():
        print(f"\n=== Dataset: {ds_name} === {ds_info}")

        if ds_name == "dana":
            metrics_bert = train_bert(ds_name, ds_info)
            print("BERT:", metrics_bert)

            results.append((ds_name, "BERT", metrics_bert))


=== Dataset: meoffendes === {'train': 'meoffendes_21_es_2_train.json', 'test': 'meoffendes_21_es_2_test.json', 'bert_model': 'dccuchile/bert-base-spanish-wwm-cased', 'num_labels': 2}

=== Dataset: hope === {'train': 'hope_24_en_4_train.json', 'test': 'hope_24_en_4_test.json', 'bert_model': 'bert-base-uncased', 'num_labels': 4}

=== Dataset: checkworthy === {'train': 'checkworthy_24_de_2_train.json', 'test': 'checkworthy_24_de_2_test.json', 'bert_model': 'bert-base-german-dbmdz-uncased', 'num_labels': 2}

=== Dataset: dana === {'train': 'dana_25_es_2_train.json', 'test': 'dana_25_es_2_test.json', 'bert_model': 'dccuchile/bert-base-spanish-wwm-cased', 'num_labels': 2}


Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/131 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-580855026.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.681900,0.501350,0.763359,0.774265,0.760961,0.759775
2,0.474700,0.542797,0.801527,0.801423,0.801423,0.801423
3,0.224000,0.708264,0.778626,0.781887,0.777285,0.777328


BERT: {'eval_loss': 0.5427970886230469, 'eval_accuracy': 0.8015267175572519, 'eval_precision': 0.8014225746268657, 'eval_recall': 0.8014225746268657, 'eval_f1': 0.8014225746268657, 'eval_runtime': 3.6367, 'eval_samples_per_second': 36.022, 'eval_steps_per_second': 4.675, 'epoch': 3.0}


In [ ]:
# =========================
# Mostrar resultados tabla

rows = []
for dataset, model, m in results:
    accuracy = m['accuracy'] if 'accuracy' in m else m.get('eval_accuracy')
    precision = m['precision'] if 'precision' in m else m.get('eval_precision')
    recall = m['recall'] if 'recall' in m else m.get('eval_recall')
    f1 = m['f1'] if 'f1' in m else m.get('eval_f1')
    rows.append({
        "Dataset": dataset,
        "Modelo": model,
        "Accuracy": accuracy,
        "Precision_macro": precision,
        "Recall_macro": recall,
        "F1_macro": f1,
    })

df = pd.DataFrame(rows)
print(df)
df.to_csv(f"{base_ds_path}classification_results.csv", index=False)

       Dataset     Modelo  Accuracy  Precision_macro  Recall_macro  F1_macro
0   meoffendes  LinearSVC  0.783399         0.758693      0.691944  0.709196
1         hope  LinearSVC  0.660210         0.576600      0.513012  0.532319
2  checkworthy  LinearSVC  0.745536         0.704929      0.696723  0.700331
3         dana  LinearSVC  0.839695         0.841281      0.840485  0.839657
4   meoffendes       BERT  0.824506         0.795982      0.775867  0.784507
5         hope       BERT  0.760291         0.682712      0.702993  0.691869
6  checkworthy       BERT  0.857143         0.833893      0.838811  0.836257
7         dana       BERT  0.801527         0.801423      0.801423  0.801423


In [ ]:

# ========================= Evaluación de explicabilidad ========================= #


In [ ]:
# Reglas para batch size
def batch_size_rules(inst_time):
    # batch size adaptativo
    if inst_time < 1.0:
       batch_size = 1000
    elif inst_time < 10.0:
        batch_size = 100
    elif inst_time < 60.0:
        batch_size = 10
    elif inst_time < 500.0:
        batch_size = 5
    else:
        batch_size = None

    print('Cero: ', inst_time, batch_size)

    return batch_size



In [ ]:
def run_xai_linearsvc(ds_name, X_test, y_test, base_ds_path):
    """
      - SHAP.LinearExplainer: n_eval = full test; Fidelity=R2_margin, Aux=MAE_margin
      - LIME: n_eval = 500 estratificado (seed=42); Fidelity=R2_local_mean, Aux=R2_local_std
    """
    import numpy as np
    import joblib
    import shap
    from lime.lime_text import LimeTextExplainer

    print(f"\n=== XAI (Linear) para {ds_name} ===")

    # Helpers

    def _batch_size_rules_or_fallback(inst_time):
        try:
            # Use the global batch_size_rules function if it exists
            bs = batch_size_rules(inst_time)
            print(f"[{ds_name}] batch_size_rules(inst_time={inst_time:.6f}) -> {bs}")
            return bs
        except NameError:
            # Fallback: same rules  (times in seconds)
            if inst_time < 1:
                bs = 1000
            elif inst_time < 10:
                bs = 100
            elif inst_time < 60:
                bs = 10
            elif inst_time < 500:
                bs = 5
            else:
                bs = None
            print(f"[{ds_name}] batch_size_rules (fallback) inst_time={inst_time:.6f} -> {bs}")
            return bs

    def _sigmoid(z):
        z = np.asarray(z, dtype=float)
        return 1.0 / (1.0 + np.exp(-z))

    def _softmax(z, axis=1):
        z = np.asarray(z, dtype=float)
        z = z - np.max(z, axis=axis, keepdims=True)
        ez = np.exp(z)
        # Avoid division by zero if sum is exactly 0.0
        return ez / (np.sum(ez, axis=axis, keepdims=True) + 1e-12)

    def make_predict_proba_fn(clf, vectorizer, n_classes):
        """Wrapper texto->proba: usa predict_proba si existe; si no, aproxima desde margin."""
        if hasattr(clf, "predict_proba"):
            print(f"[{ds_name}] predict_proba disponible en LinearSVC (calibrado).")
            def _pp(texts):
                Xv = vectorizer.transform(texts)
                return clf.predict_proba(Xv)
            return _pp
        else:
            print(f"[{ds_name}] predict_proba NO disponible; se aproximan proba desde decision_function.")
            def _pp(texts):
                Xv = vectorizer.transform(texts)
                margins = clf.decision_function(Xv)
                if n_classes == 2:
                    # Ensure margins is 1D for sigmoid in binary case
                    if np.ndim(margins) == 2 and margins.shape[1] == 1:
                        margins = margins.ravel()
                    p1 = _sigmoid(margins)
                    return np.vstack([1 - p1, p1]).T
                else:
                    return _softmax(margins, axis=1)
            return _pp

    def _normalize_shap_multiclase(shap_values, expected_value, n_classes):
        """
        Devuelve (sv_list, ev_list):
          - sv_list: lista de longitud C, cada elemento es (n_eval, n_features) para esa clase
          - ev_list: lista de longitud C con expected_value (float) por clase
        Acepta:
          * shap_values: list[ndarray] por clase  O  ndarray 3D con ejes (n,p,C) o (C,n,p)
          * expected_value: lista/ndarray por clase o escalar
        """
        print(f"[{ds_name}] Normalizando SHAP multiclase...")
        # SHAP values -> lista por clase
        if isinstance(shap_values, list):
            sv_list = shap_values
            print(f"[{ds_name}] shap_values es list con {len(sv_list)} clases.")
        else:
            sv = np.asarray(shap_values)
            print(f"[{ds_name}] shap_values array con shape={sv.shape}")
            if sv.ndim != 3:
                raise ValueError(f"Forma SHAP inesperada (ndim={sv.ndim}). Esperado 3D o lista.")
            # Casos: (n, p, C) o (C, n, p)
            if sv.shape[2] == n_classes:           # (n, p, C)
                sv_list = [sv[:, :, c] for c in range(n_classes)]
                print(f"[{ds_name}] Interpretado como (n, p, C).")
            elif sv.shape[0] == n_classes:         # (C, n, p)
                sv_list = [sv[c, :, :] for c in range(n_classes)]
                print(f"[{ds_name}] Interpretado como (C, n, p).")
            else:
                raise ValueError(f"Forma SHAP inesperada {sv.shape}. No coincide con n_clases={n_classes}.")

        # expected_value -> lista por clase
        if isinstance(expected_value, (list, tuple, np.ndarray)):
            ev_arr = np.atleast_1d(expected_value).astype(float)
            if ev_arr.shape[0] == n_classes:
                ev_list = [float(ev_arr[c]) for c in range(n_classes)]
                print(f"[{ds_name}] expected_value vector por clase con {n_classes} elementos.")
            else:
                # Fallback: if it's an array/list but size doesn't match n_classes,
                # assume it's a single value or needs to be broadcasted.
                ev_list = [float(ev_arr.squeeze())] * n_classes
                print(f"[{ds_name}] expected_value escalar replicado a {n_classes}.")
        else:
            # Scalar expected_value
            ev_list = [float(expected_value)] * n_classes
            print(f"[{ds_name}] expected_value escalar replicado a {n_classes}.")

        return sv_list, ev_list

    # --- helper para muestreo estratificado con semilla fija (mínimo cambio) ---
    def _stratified_indices(y, n, seed=42):
        y = np.asarray(y)
        rng = np.random.default_rng(seed)
        idx_per_class = {c: np.where(y == c)[0].tolist() for c in np.unique(y)}
        for lst in idx_per_class.values():
            rng.shuffle(lst)
        # cupos proporcionales (al menos 1 si hay muestras)
        total = len(y)
        alloc = {c: max(1, int(round(n * len(idx_per_class[c]) / total))) for c in np.unique(y)}
        # ajustar por exceso/defecto
        diff = n - sum(alloc.values())
        # distribuir el diff
        classes = list(idx_per_class.keys())
        i = 0
        while diff != 0 and len(classes) > 0:
            c = classes[i % len(classes)]
            if diff > 0 and alloc[c] < len(idx_per_class[c]):
                alloc[c] += 1; diff -= 1
            elif diff < 0 and alloc[c] > 1:
                alloc[c] -= 1; diff += 1
            i += 1
        # construir índices
        out = []
        for c, k in alloc.items():
            out.extend(idx_per_class[c][:k])
        rng.shuffle(out)
        return out[:n]


    def measure_time_linear_shap(explainer, instance_vectorized, batch_data_vectorized):
        import time
        start = time.perf_counter()
        # Ensure instance_vectorized is treated as a single instance (e.g., by wrapping in a list/array if needed)
        # Assuming X_test_vec[0] is already a suitable representation for a single instance
        _ = explainer.shap_values(instance_vectorized)
        inst_time = (time.perf_counter() - start)
        batch_size = _batch_size_rules_or_fallback(inst_time)
        batch_time = None
        if batch_size:
            start = time.perf_counter()
            _ = explainer.shap_values(batch_data_vectorized[:batch_size])
            batch_time = time.perf_counter() - start
        print(f"[{ds_name}] SHAP timing -> inst={inst_time:.6f}s, batch_size={batch_size}, batch_time={batch_time}")
        return inst_time, batch_time, batch_size

    def measure_time_linear_lime(lime_explainer, predict_fn, texts, num_features=10):
        import time
        if not texts:
            print(f"[{ds_name}] LIME timing -> No texts to measure.")
            return None, None, None # Or some default indicating no data

        sample_text = texts[0]
        start = time.perf_counter()
        # explain_instance works on a single string, so texts[0] is correct
        _ = lime_explainer.explain_instance(sample_text, predict_fn, num_features=num_features)
        inst_time = (time.perf_counter() - start)
        batch_size = _batch_size_rules_or_fallback(inst_time)
        batch_time = None
        if batch_size:
            start = time.perf_counter()
            # Explain each instance in the batch
            for t in texts[:batch_size]:
                _ = lime_explainer.explain_instance(t, predict_fn, num_features=num_features)
            batch_time = time.perf_counter() - start
        print(f"[{ds_name}] LIME timing -> inst={inst_time:.6f}s, batch_size={batch_size}, batch_time={batch_time}")
        return inst_time, batch_time, batch_size

    # Cargar modelo y vectorizador

    print(f"[{ds_name}] Cargando modelo y vectorizador...")
    clf = joblib.load(f"{base_ds_path}pretrained/linearsvc/{ds_name}_linearsvc_model.pkl")
    vectorizer = joblib.load(f"{base_ds_path}pretrained/linearsvc/{ds_name}_tfidf_mtc.pkl")
    print(f"[{ds_name}] Modelo: {type(clf).__name__} | Vectorizer: {type(vectorizer).__name__}")

    texts_test = list(X_test) if not isinstance(X_test, list) else X_test
    print(f"[{ds_name}] #texts_test={len(texts_test)}")

    print(f"[{ds_name}] #texts_test={len(texts_test)}")
    X_test_vec = vectorizer.transform(texts_test)
    # Use sorted(np.unique(y_test)) to handle potential missing labels in test set
    n_classes = len(np.unique(y_test))
    print(f"[{ds_name}] n_clases={n_classes} | X_test_vec shape={getattr(X_test_vec, 'shape', None)}")

    # 1) SHAP.LinearExplainer (fidelidad en margen, n_eval = full test)

    print(f"[{ds_name}] Preparando SHAP.LinearExplainer (espacio: margen)...")
    # Use a small, representative background dataset
    bg_n = min(200, X_test_vec.shape[0])
    bg_X = X_test_vec[:bg_n]
    print(f"[{ds_name}] Background size={bg_n}")
    # Initialize SHAP LinearExplainer with the model and background data
    shap_explainer = shap.LinearExplainer(clf, bg_X)

    # Measure timing
    # Ensure X_test_vec[0] is a valid single instance representation
    shap_inst_time, shap_batch_time, shap_batch_size = measure_time_linear_shap(
        shap_explainer, X_test_vec[0], X_test_vec
    )

    # Fidelity evaluation (n_eval = ALL TEST DATA or batch_size if less)
    # Adjusted n_eval to use the calculated batch_size, ensuring it's not larger than the test set
    shap_n_eval = min(X_test_vec.shape[0], shap_batch_size if shap_batch_size is not None else X_test_vec.shape[0])
    print(f"[{ds_name}] SHAP n_eval (fidelidad)={shap_n_eval}")
    X_eval_shap = X_test_vec[:shap_n_eval]  # Evaluate on the first n_eval instances

    print(f"[{ds_name}] Calculando margins y SHAP values...")
    # Get decision function values (margins) from the model
    margins = clf.decision_function(X_eval_shap)
    # Get SHAP values for the evaluation data
    shap_vals = shap_explainer.shap_values(X_eval_shap)
    # Get the expected value (base value) from the explainer
    phi0 = shap_explainer.expected_value
    print(f"[{ds_name}] margins shape={np.shape(margins)} | type(shap_vals)={type(shap_vals)} | phi0 type={type(phi0)}")

    errors = []
    r2_margin = None # Initialize R2 margin

    # Calculate fidelity based on whether it's binary or multiclass
    if np.ndim(margins) == 1:
        # --- Binary Case ---
        print(f"[{ds_name}] SHAP binario: usando clase positiva.")
        # SHAP values might be a list for binary, take the positive class (usually index 1)
        if isinstance(shap_vals, list):
            vals = shap_vals[1]
        else:
            vals = shap_vals # Should be (n_eval, n_features)

        # Handle phi0 which might be a scalar or a list for binary
        phi0_arr = np.atleast_1d(phi0).astype(float)
        phi0_scalar = float(phi0_arr[1] if phi0_arr.size > 1 else phi0_arr.squeeze())

        # Reconstruct the margin using SHAP values and the expected value
        recon = np.sum(vals, axis=1) + phi0_scalar
        # Calculate absolute error between reconstructed margin and actual margin
        err = np.abs(recon - margins.astype(float))
        errors = err

        # Calculate R² margin for binary case
        try:
            ss_res = np.sum((margins - recon)**2)
            ss_tot = np.sum((margins - np.mean(margins))**2) + 1e-12
            r2_margin = float(1.0 - ss_res/ss_tot)
        except Exception as e:
            print(f"[{ds_name}] R2 margen no calculado: {e}")
            r2_margin = None

    else:
        # --- Multiclass (OVR) Case ---
        print(f"[{ds_name}] SHAP multiclase: normalizando salida...")
        # Normalize SHAP values and expected values for multiclass
        sv_list, ev_list = _normalize_shap_multiclase(shap_vals, phi0, n_classes)
        # Find the top class for each instance based on the model's decision function
        top_c = np.argmax(margins, axis=1)
        errs = []
        # Calculate error for the top class of each instance
        for i in range(shap_n_eval):
            c = int(top_c[i])  # The class being evaluated
            vals_ic = sv_list[c][i]  # SHAP contributions for class c for instance i (n_features,)
            phi0_c = ev_list[c]      # Expected value for class c (float)
            recon_i = float(np.sum(vals_ic) + phi0_c)
            errs.append(abs(recon_i - float(margins[i, c])))
        errors = np.array(errs, dtype=float)

    # Calculate MAE and Std Dev of errors
    mae_margin = float(np.mean(errors))
    err_std = float(np.std(errors))
    print(f"[{ds_name}] SHAP fidelidad -> MAE_margin={mae_margin:.6f}, err_std={err_std:.6f}, R2_margin={r2_margin}")

    # Store SHAP results in a dictionary (row for the results table)
    row_shap = {
        "Dataset": ds_name,
        "Model": "LinearSVC",
        "Explainer": "SHAP.LinearExplainer",
        "Time_inst": float(shap_inst_time),
        "Time_batch": float(shap_batch_time) if shap_batch_time is not None else None,
        "Batch_size": int(shap_batch_size) if shap_batch_size is not None else None,
        # Fidelity metrics: MAE as primary, R2 as auxiliary
        "Fidelity": mae_margin,
        "Fidelity_metric": "MAE_margin",
        "Fidelity_aux": r2_margin # R2_margin is None for multiclass currently
    }

    # 2) LIME (R^2 local del surrogate) n_eval=500 estratificado
    print(f"[{ds_name}] Preparando LIME (R^2 local del surrogate)...")
    # Get class names for LIME (sorted unique labels)
    class_names = [str(c) for c in sorted(np.unique(y_test))]
    # Initialize LimeTextExplainer
    lime_explainer = LimeTextExplainer(class_names=class_names)

    # Create a predict_proba function wrapper for LIME
    predict_proba_fn = make_predict_proba_fn(clf, vectorizer, n_classes)

    # Measure timing for LIME
    # Pass the full list of texts for batch timing calculation
    lime_inst_time, lime_batch_time, lime_batch_size = measure_time_linear_lime(
        lime_explainer, predict_proba_fn, texts_test, num_features=10
    )

    # Fidelity evaluation (n_eval = 500 stratified or batch_size, whichever is smaller, but not more than test set)
    # Using _stratified_indices for a fixed number (500) for consistency, but respecting test set size and batch size
    lime_n_eval_target = 500 # Target number of evaluations for fidelity
    # Determine actual n_eval based on test set size and batch size
    lime_n_eval = min(len(texts_test), lime_batch_size if lime_batch_size is not None else lime_n_eval_target)
    # If the test set or batch size is smaller than the target, use all available instances up to that limit
    lime_n_eval = min(len(texts_test), lime_n_eval)

    # Get stratified indices for the evaluation set (up to lime_n_eval)
    # Ensure y_test has enough samples for stratification or handle smaller sets
    if len(y_test) >= lime_n_eval and len(np.unique(y_test)) > 1: # Only stratify if possible
       idx_eval = _stratified_indices(y_test, lime_n_eval, seed=42)
    else:
       idx_eval = list(range(lime_n_eval)) # Fallback to simple slicing

    texts_eval = [texts_test[i] for i in idx_eval]
    print(f"[{ds_name}] LIME n_eval (fidelidad)={len(texts_eval)}") # Use actual number of evaluated texts

    r2_scores = []
    # Explain each instance in the evaluation set and collect R2 scores
    for idx, txt in enumerate(texts_eval, start=1):
        # explain_instance calculates the local linear model and returns its score (R2)
        exp = lime_explainer.explain_instance(txt, predict_proba_fn, num_features=10)
        # Get the R2 score from the explanation object
        r2 = getattr(exp, "score", None)
        if r2 is None:
            # This indicates LIME was not configured to return the R2 score
            raise ValueError("LIME Explanation object does not have a 'score' attribute. Ensure LIME is configured to return R².")
        r2_scores.append(r2)
        # Print progress periodically
        if idx % max(1, len(texts_eval) // 5) == 0:
            print(f"[{ds_name}] LIME progreso {idx}/{len(texts_eval)} (R2 parcial mean={np.nanmean(r2_scores):.4f})")

    # Calculate mean and standard deviation of R2 scores
    r2_scores = np.array(r2_scores, dtype=float)
    r2_mean = float(np.nanmean(r2_scores))
    r2_std = float(np.nanstd(r2_scores))
    print(f"[{ds_name}] LIME fidelidad -> R2_local_mean={r2_mean:.6f}, R2_local_std={r2_std:.6f}")

    # Store LIME results in a dictionary (row for the results table)
    row_lime = {
        "Dataset": ds_name,
        "Model": "LinearSVC",
        "Explainer": "LimeTextExplainer",
        "Time_inst": float(lime_inst_time) if lime_inst_time is not None else None,
        "Time_batch": float(lime_batch_time) if lime_batch_time is not None else None,
        "Batch_size": int(lime_batch_size) if lime_batch_size is not None else None,
        # Fidelity metrics: R2 mean as primary, R2 std as auxiliary
        "Fidelity": r2_mean,
        "Fidelity_metric": "R2_local",
        "Fidelity_aux": r2_std # Std dev of local R2 scores
    }

    print(f"[{ds_name}] OK: filas listas (SHAP y LIME).")
    return [row_shap, row_lime]

In [ ]:
# =========================
# Correr para todos los datasets

if xai_results is None:
    xai_results = []

if False:
    for ds_name, ds_info in datasets.items():
        print(f"\n=== XAI para {ds_name} ===")
        X_train, y_train, X_test, y_test = load_json_dataset(ds_info)

        # LinearSVC
        xai_results.extend(run_xai_linearsvc(ds_name, X_test, y_test, base_ds_path))

    df_xai = pd.DataFrame(xai_results)
    print(df_xai)
    df_xai.to_csv(f"{base_ds_path}xai_results.csv", index=False)

In [ ]:
def run_xai_bert(ds_name, X_test, y_test, base_ds_path, max_length=256, inf_batch_size=32, shap_explain_bs=16):
    """
      - SHAP (Partition, logits): n_eval = 100 estratificado (seed=42); Fidelity=R2_logit, Aux=MAE_logit
      - LIME: n_eval = 200 estratificado (seed=42); Fidelity=R2_local_mean, Aux=R2_local_std
    """
    import os, re, numpy as np, torch, shap, pandas as pd
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    from lime.lime_text import LimeTextExplainer
    from time import perf_counter

    print(f"\n=== XAI (BERT) para {ds_name} ===")

    def _batch_size_rules_or_fallback(inst_time):
        try:
            bs = batch_size_rules(inst_time)
            print(f"[{ds_name}] batch_size_rules(inst_time={inst_time:.6f}) -> {bs}")
            return bs
        except NameError:
            if inst_time < 1: bs = 1000
            elif inst_time < 10: bs = 100
            elif inst_time < 60: bs = 10
            elif inst_time < 500: bs = 5
            else: bs = None
            print(f"[{ds_name}] batch_size_rules (fallback) inst_time={inst_time:.6f} -> {bs}")
            return bs

    def _to_str_list(X, batch_size=None):
        try:
            return to_str_list(X, batch_size=batch_size)
        except NameError:
            if isinstance(X, pd.Series):
                lst = X.astype(str).tolist()
            elif isinstance(X, np.ndarray):
                lst = X.astype(str).tolist()
            elif isinstance(X, list):
                lst = [str(x) for x in X]
            else:
                lst = [str(X)]
            if batch_size is not None:
                lst = lst[:batch_size]
            return lst

    def _softmax(z, axis=-1):
        z = np.asarray(z, dtype=float)
        if z.size == 0:
            return z
        z = z - np.max(z, axis=axis, keepdims=True)
        ez = np.exp(z)
        den = np.sum(ez, axis=axis, keepdims=True)
        den[den == 0.0] = 1e-12
        return ez / den

    # --- Sanitización: garantizar ≥2 tokens por texto ---
    _TOKEN_PATTERN = r"\w+|[^\w\s]"
    _token_regex = re.compile(_TOKEN_PATTERN, re.UNICODE)

    def _ensure_min2_tokens_text(s: str) -> str:
        s = "" if s is None else str(s)
        toks = _token_regex.findall(s)
        if len(toks) == 0:
            return "[UNK] [PAD]"
        if len(toks) == 1:
            return s + " [PAD]"
        return s

    # --- helper estratificado (seed fija) ---
    def _stratified_indices(y, n, seed=42):
        y = np.asarray(y)
        rng = np.random.default_rng(seed)
        idx_per_class = {c: np.where(y == c)[0].tolist() for c in np.unique(y)}
        for lst in idx_per_class.values(): rng.shuffle(lst)
        total = len(y)
        alloc = {c: max(1, int(round(n * len(idx_per_class[c]) / total))) for c in idx_per_class}
        diff = n - sum(alloc.values())
        classes = list(idx_per_class.keys()); i = 0
        while diff != 0 and len(classes) > 0:
            c = classes[i % len(classes)]
            if diff > 0 and alloc[c] < len(idx_per_class[c]): alloc[c] += 1; diff -= 1
            elif diff < 0 and alloc[c] > 1: alloc[c] -= 1; diff += 1
            i += 1
        out = []
        for c, k in alloc.items(): out.extend(idx_per_class[c][:k])
        rng.shuffle(out)
        return out[:n]

    # ---------- Carga modelo/tokenizer ----------
    candidate_paths = [
        f"{base_ds_path}pretrained/bert/{ds_name}",
        f"{base_ds_path}pretrained/bert/{ds_name}_bert",
        f"{base_ds_path}pretrained/bert/{ds_name}_model",
    ]
    model_dir = next((p for p in candidate_paths if os.path.isdir(p)), None)
    if model_dir is None:
        raise FileNotFoundError(f"[{ds_name}] No se encontró directorio BERT. Probé: {candidate_paths}")

    print(f"[{ds_name}] Cargando modelo/tokenizer desde: {model_dir}")
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
    n_classes = int(model.config.num_labels)
    print(f"[{ds_name}] num_labels={n_classes} | device={device} | max_length={max_length} | inf_batch_size={inf_batch_size}")

    @torch.no_grad()
    def predict_logits(texts):
        texts = _to_str_list(texts)
        if len(texts) == 0: return np.zeros((0, n_classes), dtype=float)
        outs = []
        for i in range(0, len(texts), inf_batch_size):
            batch = texts[i:i+inf_batch_size]
            enc = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=max_length).to(device)
            logits = model(**enc).logits.detach().cpu().numpy()
            outs.append(logits)
        return np.vstack(outs) if outs else np.zeros((0, n_classes), dtype=float)

    def predict_proba(texts):
        logits = predict_logits(texts)
        return _softmax(logits, axis=1)

    # Textos de test con ≥2 tokens garantizados
    texts_test = _to_str_list(X_test)
    texts_test = [_ensure_min2_tokens_text(t) for t in texts_test]
    print(f"[{ds_name}] #texts_test={len(texts_test)}")

    # ---------- SHAP (Partition, logits) ----------
    print(f"[{ds_name}] Preparando SHAP.PartitionExplainer (espacio: logits)...")
    masker = shap.maskers.Text(_TOKEN_PATTERN)  # tokeniza por regex (palabras y signos)
    print(f"[{ds_name}] masker=Text(regex={_TOKEN_PATTERN})")
    explainer_shap = shap.Explainer(predict_logits, masker, algorithm="partition",
                                    output_names=[str(i) for i in range(n_classes)])

    # timings intactos
    t0 = perf_counter(); _ = explainer_shap([texts_test[0]]); shap_inst_time = perf_counter() - t0
    shap_batch_size = _batch_size_rules_or_fallback(shap_inst_time)
    shap_batch_time = None
    if shap_batch_size:
        t0 = perf_counter(); _ = explainer_shap(texts_test[:shap_batch_size]); shap_batch_time = perf_counter() - t0
    print(f"[{ds_name}] SHAP timing -> inst={shap_inst_time:.6f}s, batch_size={shap_batch_size}, batch_time={shap_batch_time}")

    # n_eval fijo y estratificado
    shap_n_eval = min(len(texts_test), 100)
    idx_eval_shap = _stratified_indices(y_test, shap_n_eval, seed=42)
    texts_eval = [texts_test[i] for i in idx_eval_shap]
    print(f"[{ds_name}] SHAP n_eval (fidelidad)={shap_n_eval} (estratificado, seed=42)")

    logits_true = predict_logits(texts_eval)
    if logits_true.shape[0] == 0:
        raise RuntimeError(f"[{ds_name}] predict_logits devolvió (0, C) para textos no vacíos.")
    top_c = np.argmax(logits_true, axis=1)

    # explicar por bloques con tolerancia
    exps, ok_idx = [], []
    for i in range(0, shap_n_eval, shap_explain_bs):
        chunk_idx = list(range(i, min(i+shap_explain_bs, shap_n_eval)))
        chunk_texts = [texts_eval[k] for k in chunk_idx]
        try:
            e_chunk = explainer_shap(chunk_texts)
            exps.extend(list(e_chunk)); ok_idx.extend(chunk_idx)
            print(f"[{ds_name}] SHAP progreso {chunk_idx[-1]+1}/{shap_n_eval}")
        except Exception as e:
            print(f"[{ds_name}] SHAP batch fallo en [{i}:{i+shap_explain_bs}]: {e} -> instancia-por-instancia")
            for k in chunk_idx:
                try:
                    e1 = explainer_shap([texts_eval[k]])
                    exps.extend(list(e1)); ok_idx.append(k)
                except Exception as e2:
                    print(f"[{ds_name}] SHAP omitida inst {k}: {e2}")

    if len(ok_idx) == 0:
        raise RuntimeError(f"[{ds_name}] No se pudo generar ninguna explicación SHAP.")

    # Reconstrucción de logit para clase top
    y_true, y_hat, abs_errors = [], [], []
    for cnt, k in enumerate(ok_idx, start=1):
        exp = exps[cnt-1]
        vals = np.asarray(exp.values)
        # Normalizar a (tokens, C)
        if vals.ndim == 2 and vals.shape[1] == n_classes:
            token_class = vals
        elif vals.ndim == 2 and n_classes == 2 and vals.shape[1] == 1:
            token_class = np.hstack([-vals, vals])  # (tokens, 2)
        elif vals.ndim == 1 and n_classes == 2:
            token_class = np.vstack([-vals, vals]).T
        else:
            raise ValueError(f"[{ds_name}] SHAP: valores con forma {vals.shape}, inesperada para n_classes={n_classes}.")

        base_vals = np.asarray(exp.base_values)
        if base_vals.ndim == 1 and base_vals.shape[0] == n_classes:
            base = base_vals
        else:
            base = np.ravel(base_vals)
            if base.shape[0] != n_classes:
                base = np.full((n_classes,), float(base_vals if np.isscalar(base_vals) else np.mean(base_vals)))

        c = int(top_c[k])
        recon_c = float(token_class[:, c].sum() + float(base[c]))
        y_hat.append(recon_c); y_true.append(float(logits_true[k, c]))
        abs_errors.append(abs(recon_c - float(logits_true[k, c])))

        if cnt % max(1, len(ok_idx)//5) == 0:
            print(f"[{ds_name}] SHAP recon progreso {cnt}/{len(ok_idx)} | err_abs_mean={np.mean(abs_errors):.4f}")

    y_true = np.array(y_true, dtype=float); y_hat = np.array(y_hat, dtype=float)
    abs_errors = np.array(abs_errors, dtype=float)

    mae_logit = float(np.mean(abs_errors))
    try:
        ss_res = np.sum((y_true - y_hat)**2); ss_tot = np.sum((y_true - np.mean(y_true))**2) + 1e-12
        r2_logit = float(1.0 - ss_res/ss_tot)
    except Exception as e:
        print(f"[{ds_name}] R2_logit no calculado: {e}"); r2_logit = None

    print(f"[{ds_name}] SHAP fidelidad -> R2_logit={r2_logit}, MAE_logit={mae_logit:.6f}")
    row_shap = {
        "Dataset": ds_name, "Model": "BERT", "Explainer": "SHAP.PartitionExplainer",
        "Time_inst": float(shap_inst_time), "Time_batch": float(shap_batch_time) if shap_batch_time is not None else None,
        "Batch_size": int(shap_batch_size) if shap_batch_size is not None else None,
        "Fidelity": r2_logit, "Fidelity_metric": "R2_logit", "Fidelity_aux": mae_logit
    }

    # ---------- LIME (R2_local) ----------
    print(f"[{ds_name}] Preparando LIME (R2 local del surrogate)...")
    class_names = [str(c) for c in sorted(np.unique(y_test))]
    lime_explainer = LimeTextExplainer(class_names=class_names)

    # timings intactos
    t0 = perf_counter(); _ = lime_explainer.explain_instance(texts_test[0], predict_proba, num_features=10)
    lime_inst_time = perf_counter() - t0
    lime_batch_size = _batch_size_rules_or_fallback(lime_inst_time)
    lime_batch_time = None
    if lime_batch_size:
        t0 = perf_counter()
        for t in texts_test[:lime_batch_size]:
            _ = lime_explainer.explain_instance(t, predict_proba, num_features=10)
        lime_batch_time = perf_counter() - t0
    print(f"[{ds_name}] LIME timing -> inst={lime_inst_time:.6f}s, batch_size={lime_batch_size}, batch_time={lime_batch_time}")

    # n_eval fijo y estratificado
    lime_n_eval = min(len(texts_test), 200)
    idx_eval_lime = _stratified_indices(y_test, lime_n_eval, seed=42)
    texts_eval = [texts_test[i] for i in idx_eval_lime]
    print(f"[{ds_name}] LIME n_eval (fidelidad)={lime_n_eval} (estratificado, seed=42)")

    r2_scores = []
    for i, txt in enumerate(texts_eval, start=1):
        exp = lime_explainer.explain_instance(txt, predict_proba, num_features=10)
        r2 = getattr(exp, "score", None)
        if r2 is None:
            raise ValueError("LIME Explanation no tiene 'score'. Configura LIME para devolver R².")
        r2_scores.append(r2)
        if i % max(1, lime_n_eval // 5) == 0:
            print(f"[{ds_name}] LIME progreso {i}/{lime_n_eval} (R2 parcial mean={np.nanmean(r2_scores):.4f})")

    r2_scores = np.array(r2_scores, dtype=float)
    r2_mean = float(np.nanmean(r2_scores)); r2_std = float(np.nanstd(r2_scores))
    print(f"[{ds_name}] LIME fidelidad -> R2_local_mean={r2_mean:.6f}, R2_local_std={r2_std:.6f}")
    row_lime = {
        "Dataset": ds_name, "Model": "BERT", "Explainer": "LimeTextExplainer",
        "Time_inst": float(lime_inst_time), "Time_batch": float(lime_batch_time) if lime_batch_time is not None else None,
        "Batch_size": int(lime_batch_size) if lime_batch_size is not None else None,
        "Fidelity": r2_mean, "Fidelity_metric": "R2_local", "Fidelity_aux": r2_std
    }

    print(f"[{ds_name}] OK: filas listas (SHAP y LIME).")
    return [row_shap, row_lime]


In [ ]:
# =========================
# Correr para todos los datasets

xai_results = []
if xai_results is None:
    xai_results = []

for ds_name, ds_info in datasets.items():
    print(f"\n=== XAI para {ds_name} ===")
    X_train, y_train, X_test, y_test = load_json_dataset(ds_info)

    # BERT (timings intactos; fidelidad estandarizada)
    xai_results.extend(run_xai_bert(ds_name, X_test, y_test, base_ds_path))

df_xai = pd.DataFrame(xai_results)
print(df_xai)
df_xai.to_csv(f"{base_ds_path}xai_results_bert_solo_dana_today.csv", index=False)


* LinearSVC+LIME → `Fidelity=R2_local_mean`, `Aux=R2_local_std`.
* LinearSVC+SHAP → `Fidelity=R2_margin`, `Aux=MAE_margin`.
* BERT+LIME → `Fidelity=R2_local_mean`, `Aux=R2_local_std`.
* BERT+SHAP → `Fidelity=R2_logit`, `Aux=MAE_logit`.

>

* **LinearSVC + LIME (500)**: más estable y comparable; más coste en fidelidad por evaluar más instancias.
* **LinearSVC + SHAP (todo test)**: máxima precisión (R²≈1, MAE≈0) con coste mínimo; mejora la solidez del reporte.
* **BERT + LIME (200)**: R² local más estable que con 10/100 muestras; coste extra en fidelidad.
* **BERT + SHAP (100)**: suficiente para un R²\_logit representativo, con coste **menor** que antes si intentabas evaluar cientos/miles.

